In [1]:
%%capture
# Install necessary libraries for the project
!pip install datasets
!pip install transformers[torch]
!pip install googletrans==4.0.0-rc1

In [2]:
import pandas as pd
import time

from tqdm import tqdm  # Import the tqdm library for progress bars
from googletrans import Translator  # Import the Translator class from the googletrans library

# Load sample submission data
sample = pd.read_csv("https://raw.githubusercontent.com/Bratuz/Jaeger/master/detecting-french-texts-difficulty-level-2023/Data/sample_submission.csv")
# Load training dataset
train_dataset = pd.read_csv("https://raw.githubusercontent.com/Bratuz/Jaeger/master/detecting-french-texts-difficulty-level-2023/Data/training_data.csv")
# Load test dataset
test_dataset = pd.read_csv("https://raw.githubusercontent.com/Bratuz/Jaeger/master/detecting-french-texts-difficulty-level-2023/Data/unlabelled_test_data.csv")

In [3]:
# Define a function (back_translate) with options for the target language, maximum retries, and sleep time between attempts.
def back_translate(text, target_language='fr', max_retries=3, sleep_time=5):
    retries = 0
    while retries < max_retries:
        try:
            # Initial translation to the target language
            translator = Translator()
            translated_text = translator.translate(text, dest=target_language).text

            # Back-translation to English
            back_translated_text_en = translator.translate(translated_text, dest='en').text

            # Second back-translation to the original target language (French in this case)
            final_back_translated_text = translator.translate(back_translated_text_en, dest=target_language).text

            return final_back_translated_text
        except Exception as e:
            print(f"Error: {e}. Retrying...")
            retries += 1
            time.sleep(sleep_time * retries)

    print("Maximum number of retries reached. Returning to the original text.")
    return text

# Create a new DataFrame to store the translations
new_df = pd.DataFrame()

# Copy 'sentence' and 'difficulty' columns into the new DataFrame
new_df['original_sentence'] = train_dataset['sentence']
new_df['difficulty'] = train_dataset['difficulty']

# Apply the back translation function to the 'sentence' column with a progress bar
tqdm.pandas()
new_df['sentence'] = train_dataset['sentence'].progress_apply(back_translate)

 48%|████▊     | 2292/4800 [1:10:56<1:21:54,  1.96s/it]

Error: The read operation timed out. Retrying...


 48%|████▊     | 2295/4800 [1:11:13<2:32:21,  3.65s/it]

Error: The read operation timed out. Retrying...


 49%|████▉     | 2341/4800 [1:13:13<2:08:21,  3.13s/it]

...

 78%|███████▊  | 3764/4800 [1:59:09<44:11,  2.56s/it]

Error: The read operation timed out. Retrying...
Error: The read operation timed out. Retrying...


 92%|█████████▏| 4419/4800 [2:20:06<12:22,  1.95s/it]

Error: Invalid input ConnectionInputs.RECV_PING in state ConnectionState.CLOSED. Retrying...


100%|██████████| 4800/4800 [2:31:54<00:00,  1.90s/it]


In [4]:
# Drop the 'original_sentence' column from the DataFrame
new_df.drop('original_sentence', axis=1, inplace=True)

In [5]:
# Switch the order of columns 'sentence' and 'difficulty'
new_df = new_df[['sentence', 'difficulty'] + [col for col in new_df.columns if col not in ['sentence', 'difficulty']]]


In [6]:
# Display the first few rows of the DataFrame
new_df.head()

,sentence,difficulty
0,Les coûts kilométriques réels peuvent diverger...,C1
1,Le bleu est ma couleur préférée mais je n'aime...,A1
2,Le test de niveau en français est sur le site ...,A1
3,Votre mari est-il également de Boston?,A1
4,"Dans les écoles de commerce, dans les couloirs...",B1


In [7]:
# Save translation to a CSV file
new_df.to_csv("translate.csv", index=False)


In [8]:
from google.colab import files

# Download the CSV file "translate.csv" from Google Colab
files.download("translate.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>